In [1]:
import xarray as xr
import numpy as np
import pandas as pd

This script computes spatial similarity between ERA5 and modelled CCA-generated coupled sea ice concentration and surface air temperature  patterns using area-weighted cosine similarity method. ERA5 patterns associated with CO₂, IPO, and AMO are interpolated to the model grid (here, CanESM2), and appropriate sign flips are applied based on expected physical alignment. The method ensures consistent handling of masked data and grid weighting. The approach can be readily applied to the rest of the climate models' data by replacing the input datasets and maintaining the pattern association and preprocessing conventions.

In [2]:
# Area-weighted cosine similarity
#SAT 
def simple_area_weighted_cosine(a, b, lat):
    weights = np.cos(np.radians(lat))[:, np.newaxis]
    weights_2d = np.broadcast_to(weights, a.shape)
    dot = np.sum(a * b * weights_2d)
    norm_a = np.sqrt(np.sum((a**2) * weights_2d))
    norm_b = np.sqrt(np.sum((b**2) * weights_2d))
    return dot / (norm_a * norm_b)

#SIC 
def hybrid_similarity(obs, model, lat, threshold=1.0):
    weights = np.cos(np.radians(lat))[:, np.newaxis]
    weights_2d = np.broadcast_to(weights, obs.shape)
    mask = (np.abs(obs) > threshold) | (np.abs(model) > threshold)
    obs_masked = np.where(mask, obs, np.nan)
    model_masked = np.where(mask, model, np.nan)

    def normalize(x):
        x_flat = x[~np.isnan(x)]
        return (x - np.nanmean(x_flat)) / np.nanstd(x_flat)

    obs_norm = normalize(obs_masked)
    model_norm = normalize(model_masked)
    model_norm_flipped = -1 * model_norm

    def cosine_sim(a, b):
        a_flat = a[~np.isnan(a) & ~np.isnan(b)]
        b_flat = b[~np.isnan(a) & ~np.isnan(b)]
        w_flat = weights_2d[~np.isnan(a) & ~np.isnan(b)]
        dot = np.sum(a_flat * b_flat * w_flat)
        norm_a = np.sqrt(np.sum((a_flat**2) * w_flat))
        norm_b = np.sqrt(np.sum((b_flat**2) * w_flat))
        return dot / (norm_a * norm_b)

    cos1 = cosine_sim(obs_norm, model_norm)
    cos2 = cosine_sim(obs_norm, model_norm_flipped)
    return {"cosine": cos2 if abs(cos2) > abs(cos1) else cos1}

# Rename coordinates
def safe_rename(ds):
    coords = ds.coords
    rename_dict = {}
    if 'latitude' in coords:
        rename_dict['latitude'] = 'lat'
    if 'longitude' in coords:
        rename_dict['longitude'] = 'lon'
    return ds.rename(rename_dict)

# Load model CCA patterns
sic = xr.open_dataset("SIC_CanESM2_CCA123.nc")
sat = xr.open_dataset("SAT_CanESM2_CCA123.nc").rename({
    'sic1': 'sat1', 'sic2': 'sat2', 'sic3': 'sat3'
})

# Load ERA5 CCA patterns
obs_sic = {
    "CO2": xr.open_dataset("SIC_OBS_CCA1_CO2.nc")["sic1"],
    "AMO": -1 * xr.open_dataset("SIC_OBS_CCA2_AMO.nc")["sic2"],
    "IPO": 1 * xr.open_dataset("SIC_OBS_CCA3_IPO.nc")["sic3"]
}
obs_sat = {
    "CO2": xr.open_dataset("T2M_Comp_CO2.nc")["T2M"],
    "AMO": xr.open_dataset("T2M_diff_PC2_AMO_compoz.nc")["T2M"],
    "IPO": xr.open_dataset("T2M_diff_composite_Antarctica_PC3_IPO.nc")["T2M"]
}

# Standardize coordinate names
obs_sic = {key: safe_rename(ds) for key, ds in obs_sic.items()}
obs_sat = {key: safe_rename(ds) for key, ds in obs_sat.items()}

# Interpolate observations to CanESM2 grid
obs_sic_interp = {key: ds.interp(lat=sic.lat, lon=sic.lon, method="nearest") for key, ds in obs_sic.items()}
obs_sat_interp = {key: ds.interp(lat=sat.lat, lon=sat.lon, method="nearest") for key, ds in obs_sat.items()}

# Flip model components to highlight the corresponding phase of the three drivers
sic2_flipped = -1 * sic["sic2"]
sic3_flipped = -1 * sic["sic3"]
sat2_flipped = -1 * sat["sat2"]
sat3_flipped = -1 * sat["sat3"]

# --- Compute Cosine Similarities ---

# SIC cosine similarity
sic_cosine = [
    hybrid_similarity(obs_sic_interp["CO2"].values, sic["sic1"].values, sic["lat"].values, threshold=1.0)["cosine"],
    hybrid_similarity(obs_sic_interp["IPO"].values, sic2_flipped.values, sic["lat"].values)["cosine"],
    hybrid_similarity(obs_sic_interp["AMO"].values, sic3_flipped.values, sic["lat"].values)["cosine"]
]

# SAT cosine similarity 
sat_cosine = [
    simple_area_weighted_cosine(obs_sat_interp["CO2"].values, sat["sat1"].values, sat["lat"].values),
    simple_area_weighted_cosine(obs_sat_interp["IPO"].values, sat2_flipped.values, sat["lat"].values),
    simple_area_weighted_cosine(obs_sat_interp["AMO"].values, sat3_flipped.values, sat["lat"].values)
]

# --- Results ---

df = pd.DataFrame({
    "Model": "CanESM2",
    "Pattern": ["CO2", "IPO", "AMO"],
    "SIC": sic_cosine,
    "SAT": sat_cosine
})

print(df)

     Model Pattern       SIC       SAT
0  CanESM2     CO2  0.394272  0.719655
1  CanESM2     IPO  0.449567  0.523698
2  CanESM2     AMO  0.057840  0.032349
